In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Downstream_Dataset(path=data_file))
    d = Downstream_Dataset(path=data_file)
    print(d.labels.shape)
f.close()


dataset = torch.utils.data.ConcatDataset(datasets_list)
data_len = len(dataset)
print("dataset len is", len(dataset))

train_len = int(data_len*0.6)
val_len = data_len - train_len

training_set, validation_set = torch.utils.data.random_split(dataset, [train_len, val_len])


Loading Data
Processing MouseCKA1_030515_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
Processing MouseCKA1_030615_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
Processing MouseCKL1_062514_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2

In [3]:
smallest_class = smallest_class_len(training_set, 3)
num_samples=[]
temp=1
while temp < smallest_class:
    num_samples.append(temp)
    temp*=10
num_samples.append(None)
print(num_samples)

1
1
0
1
2
0
0
0
2
2
1
2
0
1
0
2
0
0
1
2
2
0
2
2
1
0
2
0
1
2
1
1
0
0
1
2
2
2
2
2
2
2
0
0
1
2
1
0
2
1
0
2
2
2
2
1
2
2
2
2
0
2
2
2
1
0
1
0
0
0
0
0
2
2
2
2
1
2
2
2
0
1
2
0
0
0
2
0
1
2
0
2
0
0
1
1
2
1
1
2
1
0
0
1
0
2
2
2
0
0
2
0
1
2
2
2
2
1
0
1
2
0
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
2
0
0
1
2
2
1
2
2
2
1
1
2
2
0
0
0
2
2
2
2
0
2
2
2
2
0
1
0
0
2
2
0
2
1
2
0
1
1
1
1
1
0
2
1
0
1
2
1
2
2
1
2
2
2
1
0
0
2
2
2
0
2
1
2
2
1
2
2
1
2
2
57
50
109
[1, 10, None]


In [4]:
def train_different_classes(model_path, tset, vset, epochs, sample_list):
    outList=[]
    for i in sample_list:
        print(i)
        outList.append(train_end_to_end(model_path, tset, vset, i, epochs, 3))
        print(outList)
    return outList

In [5]:
RP_vals = train_different_classes("RP_stagernet.pth", training_set, validation_set, 100, num_samples)

1
len of the dataloader is: 1
Start Training
[0.5416666666666666]
10
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888]
None
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888, 0.8611111111111112]


In [6]:
print(RP_vals)

[0.5416666666666666, 0.8263888888888888, 0.8611111111111112]


In [35]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    d = Downstream_Dataset(path=data_file)
    print(d.labels.shape)
    print(d.data.shape)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
    
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(group.shape)


from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()
logo.get_n_splits(x_vals, y_vals, groups)


for train_index, test_index in logo.split(x_vals, y_vals, groups):
    from torch import Tensor
    from torch.utils.data import DataLoader, TensorDataset
    training_set = TensorDataset(Tensor(x_vals[train_index]), Tensor(y_vals[train_index]))
    validation_set = TensorDataset(Tensor(x_vals[test_index]), Tensor(y_vals[test_index]))
    print(len(training_set))


Loading Data
Processing MouseCKA1_030515_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
(40, 3000, 16)
Processing MouseCKA1_030615_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
(40, 3000, 16)
Processing MouseCKL1_062514_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
(40, 3000, 16)
Processing MouseCKB9_022715_HCOFTS
labels shape (40,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
data shape (40, 3000, 16)
removed 0 unknown entries
(40,)
(4